In [44]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

import pandas as pd
import time

In [45]:

# creating an empty dataframe 
variables = {'cafe_name': str(), 'city': str(), 'province': str(), 'phone_number': str(), 'cost': int(), 'work_start': time,
'work_end': time, 'cafe_address': str(), 'hookah': bool(), 'internet': bool(), 'delivery': bool(), 'smoking': bool(), 'open_space': bool(), 'live_music': bool(), 'parking': bool(),
'pos': bool(), 'food_quality': int(), 'service': int(), 'cost_value': int(), 'environment': int(), 'follower': int(), 'lat': float(), 'lon': float()}
df_cafe = pd.DataFrame(variables, index=[])
df_cafe.dtypes

cafe_name        object
city             object
province         object
phone_number     object
cost              int64
work_start       object
work_end         object
cafe_address     object
hookah             bool
internet           bool
delivery           bool
smoking            bool
open_space         bool
live_music         bool
parking            bool
pos                bool
food_quality      int64
service           int64
cost_value        int64
environment       int64
follower          int64
lat             float64
lon             float64
dtype: object

In [46]:
# function for convert to time
def changetime(time : 'str'):
    time = time.strip()
    if time == '24':
        time = '00'

    if ':' in time:
        newtime = datetime.strptime(time.strip(), '%H:%M').time()
    else:
        newtime = datetime.strptime(time.strip(), '%H').time()
    return newtime

In [75]:

# function for extract data from a cafe
def Data_cafe(URL, df_cafe, city):
    try:
        resp = requests.get(URL)
        html = resp.text
        soup = BeautifulSoup(html, "html.parser")

        cafe_name = soup.find('div', {'class': "venue-name-box"}).find('h1',{'property':'name'}).text

        province = ((soup.find('span', {'property': "address"}).text).split('،')[0].strip())[:30]

        phone_number = str(soup.find('span', {'property': "telephone"}).text.strip())

        cost = int(soup.find('div', {'class': "price-class"}).find_all('span')[-1].text)
    
        time_work = ((soup.find('ul', {'class': "infolist"}).find_all('li')[2].text).split()[3])
        if '/' in time_work:
            time_work = time_work.split('/')
            work_start = changetime(time_work[0].split('-')[0])
            work_end =changetime(time_work[1].split('-')[1])
        else:
            time_work = time_work.split('-')
            work_start = changetime(time_work[0])
            work_end =changetime(time_work[1])


        cafe_address = soup.find('span', {'property': "address"}).text.strip()

        features = soup.find('div', {'class': "venue-features-box"}).text

        hookah = False
        internet = False
        delivery = False
        smoking = False
        open_space = False
        live_music = False
        parking = False
        pos = False

        if "ارسال رایگان" in features:
            delivery = True
        if "دستگاه کارت خوان" in features:
            pos = True
        if "اینترنت رایگان" in features:
            internet = True
        if "سیگار" in features:
            smoking = True
        if "موسیقی زنده" in features:
            live_music = True
        if "قلیان" in features:
            hookah = True
        if "فضای باز" in features:
            open_space = True
        if "پارکینگ" in features:
            parking = True    

        food_quality = int((soup.find('ul', {'class': "rates-list"}) \
            .find_all('li')[0].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])

        service = int((soup.find('ul', {'class': "rates-list"}) \
            .find_all('li')[1].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])

        cost_value = int((soup.find('ul', {'class': "rates-list"}) \
            .find_all('li')[2].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])
        
        enviornment = int((soup.find('ul', {'class': "rates-list"}) \
            .find_all('li')[3].find('div',{'class': 'rate-it'})).attrs['data-rateit-value'])
        
        #followers find
        follower = soup.find('span', {'class': 'follow-count'})
        follower = follower.text.strip().replace('دنبال کننده', '')
        follower = int(follower.strip())
     

        #lat and lon
        location = soup.find('div', {'class': 'map-container'}).find('a')
        location = location.attrs['href'].strip().replace('http://www.google.com/maps/place/', '')
        location = location.split(',')
        lat = float(location[0])
        lon = float(location[1])
       

        new_raw = {'cafe_name': cafe_name, 'city': city, 'province': province, 'phone_number': phone_number, 'cost': cost,
        'work_start': work_start,'work_end': work_end, 'cafe_address': cafe_address, 'hookah': hookah, 'internet': internet,
        'delivery': delivery, 'smoking': smoking, 'open_space': open_space, 'live_music': live_music, 'parking': parking,
        'pos': pos, 'food_quality': food_quality, 'service':service, 'cost_value':cost_value, 'environment': enviornment,
        'follower': follower,'lat': lat, 'lon': lon}
        df_cafe = df_cafe.append(new_raw, ignore_index = True)
        
        
        
    except:
        pass
    return df_cafe


In [48]:
#function for extract links of cafe
def Cafes_link(URL, cafes_links, city):
    resp = requests.get(URL)
    html = resp.text
    soup = BeautifulSoup(html, "html.parser")
    
    for item in soup.find_all('div',{'class': 'restaurant-list-items span-4 th-span-6 mn-span-12'}):
        link = item.find('a', {'class': 'restaurant-link'}).attrs['href'] 
        if link not in cafes_links[city]:
            cafes_links[city].append(link)

    
    return cafes_links

In [52]:
#extract  cities_name and pages_number of them
URL = "https://fidilio.com/coffeeshops/in/tehran/"
resp = requests.get(URL)
html = resp.text
soup = BeautifulSoup(html, "html.parser")


cities = soup \
.find("select", {"id": "cityClass"}) \
.find_all("option")
cities_list = []
for option in cities:
    cities_list.append(option.attrs['value'])
print(cities_list)


CityPage = {}
for c in set(cities_list):
    URL = f"https://fidilio.com/coffeeshops/in/{c}/"
    resp = requests.get(URL)
    html = resp.text
    soup = BeautifulSoup(html, "html.parser")
    
    last_page = soup \
    .find("div", {"id": "container"}) \
    .find_all('a')[-1].attrs['href']

    
    URL = last_page
    resp = requests.get(URL)
    html = resp.text
    soup = BeautifulSoup(html, "html.parser")

    pages_max = soup \
    .find("div", {"id": "container"}) \
    .find_all('a')[-3].string
    CityPage[c] = int(pages_max)
    


print(CityPage)

['tehran', 'shiraz', 'isfahan', 'mashhad', 'tabriz', 'kish', 'tehran', 'ghom', 'arak', 'ahwaz', 'sabzevar', 'urmia', 'zanjan', 'qazvin', 'hamedan', 'karaj', 'kerman', 'bandarabbas']
{'tabriz': 1, 'urmia': 1, 'ahwaz': 2, 'zanjan': 1, 'tehran': 48, 'kish': 1, 'hamedan': 1, 'kerman': 1, 'karaj': 3, 'qazvin': 1, 'arak': 1, 'isfahan': 2, 'bandarabbas': 1, 'sabzevar': 1, 'shiraz': 2, 'ghom': 1, 'mashhad': 1}


In [54]:
#extract all page_links of each city
cafes_links = {}
for city,pages in CityPage.items():
    if city in cafes_links.keys():
        pass
    else:
        cafes_links[city] = []
    print(city)
    if pages > 1:
        for num in range(pages):
            URL = f"https://fidilio.com/coffeeshops/in/{city}/?p={num}"
            print(URL)
            Cafes_link(URL,cafes_links,city)
    else:
        URL = f"https://fidilio.com/coffeeshops/in/{city}/" 
        Cafes_link(URL,cafes_links,city)

tabriz
urmia
ahwaz
https://fidilio.com/coffeeshops/in/ahwaz/?p=0
https://fidilio.com/coffeeshops/in/ahwaz/?p=1
zanjan
tehran
https://fidilio.com/coffeeshops/in/tehran/?p=0
https://fidilio.com/coffeeshops/in/tehran/?p=1
https://fidilio.com/coffeeshops/in/tehran/?p=2
https://fidilio.com/coffeeshops/in/tehran/?p=3
https://fidilio.com/coffeeshops/in/tehran/?p=4
https://fidilio.com/coffeeshops/in/tehran/?p=5
https://fidilio.com/coffeeshops/in/tehran/?p=6
https://fidilio.com/coffeeshops/in/tehran/?p=7
https://fidilio.com/coffeeshops/in/tehran/?p=8
https://fidilio.com/coffeeshops/in/tehran/?p=9
https://fidilio.com/coffeeshops/in/tehran/?p=10
https://fidilio.com/coffeeshops/in/tehran/?p=11
https://fidilio.com/coffeeshops/in/tehran/?p=12
https://fidilio.com/coffeeshops/in/tehran/?p=13
https://fidilio.com/coffeeshops/in/tehran/?p=14
https://fidilio.com/coffeeshops/in/tehran/?p=15
https://fidilio.com/coffeeshops/in/tehran/?p=16
https://fidilio.com/coffeeshops/in/tehran/?p=17
https://fidilio.com/c

In [55]:
#number of all cafes
summ = 0
for i,j in cafes_links.items():
    summ += len(j)

print(summ)

1230


In [56]:
#send links to Data_cafe and at the end recive dataframe 
url ='https://fidilio.com'

for city, links in cafes_links.items():
    for link in links:
        URL = f"{url}{link}"
        df_cafe = Data_cafe(URL, df_cafe, city)

In [57]:
df_cafe

,cafe_name,city,province,phone_number,cost,work_start,work_end,cafe_address,hookah,internet,...,live_music,parking,pos,food_quality,service,cost_value,environment,follower,lat,lon
0,رولی (تبریز),tabriz,اول عباسی,04135243397,2,11:00:00,23:00:00,اول عباسی، به طرف منصور,False,False,...,False,False,True,0,0,0,0,0,38.080036,46.308585
1,لاله زار (تبریز),tabriz,چایکنار,04132820831,2,11:00:00,23:00:00,چایکنار، بعد از کارواش شهرداری، نرسیده به بلوا...,False,False,...,False,False,True,0,0,0,0,0,38.097036,46.271099
2,سانیز,tabriz,پل سنگی,04133346101,2,11:00:00,23:00:00,پل سنگی، چایکنار,False,False,...,False,False,True,0,0,0,0,0,38.077004,46.316582
3,آسا,tabriz,ولیعصر,04133314649,2,08:30:00,23:30:00,ولیعصر، خیابان شریعتی، نرسیده به میدان بهارستا...,False,False,...,False,False,True,0,0,0,0,0,38.064740,46.359343
4,کلبه (تبریز),tabriz,ولیعصر,04133249214,2,10:00:00,23:30:00,ولیعصر، نگین پارک,False,True,...,False,False,True,0,0,0,0,0,38.066835,46.351025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1189,لئون,mashhad,بزرگراه کلانتری,09123848002,2,10:00:00,23:45:00,بزرگراه کلانتری ، مجموعه توریستی آبادگران ، ای...,False,True,...,False,False,True,0,0,0,0,0,36.268532,59.551864
1190,رترو (مشهد),mashhad,بلوار پیروزی,35098085,2,13:30:00,00:00:00,بلوار پیروزی ، بین پیروزی 79 و 81 ، شماره 79,False,True,...,False,False,True,0,0,0,0,0,36.332153,59.465501
1191,1860,mashhad,هاشمیه 91/5,38814117,2,17:00:00,23:30:00,هاشمیه 91/5 ، خیابان سوم سمت چپ ، قطعه آخر,False,True,...,False,False,True,4,4,5,4,0,36.301612,59.485484
1192,نبات,mashhad,خیابان سلمان فارسی,38443918,2,08:00:00,00:00:00,خیابان سلمان فارسی ، نرسیده به خیابان فلسطین ،...,False,True,...,False,False,True,5,4,4,3,0,36.305131,59.567860


In [62]:

df_cafe_1 = df_cafe.copy()

df_cafe_1.describe()

,cost,food_quality,service,cost_value,environment,follower,lat,lon
count,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.0,1194.000000,1194.000000
mean,1.980737,2.118928,2.046901,2.157454,2.010050,0.0,35.315069,51.438483
std,0.330811,2.158651,1.950643,2.021344,1.910372,0.0,1.650512,1.408558
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,26.517163,45.038686
25%,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,35.707550,51.364851
50%,2.000000,3.000000,2.000000,3.000000,2.000000,0.0,35.743834,51.413290
75%,2.000000,4.000000,4.000000,4.000000,4.000000,0.0,35.782519,51.447233
max,4.000000,31.000000,5.000000,5.000000,5.000000,0.0,38.097036,59.572999


In [63]:
#fix food_quality where value is bigger than 5
df_cafe_1 =  df_cafe_1.drop_duplicates(keep='first').reset_index()
df_cafe_1.dtypes
df_cafe_1.groupby('food_quality').count()
def f(x):
    if x>5:
        return 5
df_cafe_1.loc[(df_cafe_1['food_quality'] >= 5),'food_quality'] = 5

df_cafe_1.describe()

,index,cost,food_quality,service,cost_value,environment,follower,lat,lon
count,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.000000,1194.0,1194.000000,1194.000000
mean,596.500000,1.980737,2.094640,2.046901,2.157454,2.010050,0.0,35.315069,51.438483
std,344.822418,0.330811,1.986186,1.950643,2.021344,1.910372,0.0,1.650512,1.408558
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,26.517163,45.038686
25%,298.250000,2.000000,0.000000,0.000000,0.000000,0.000000,0.0,35.707550,51.364851
50%,596.500000,2.000000,3.000000,2.000000,3.000000,2.000000,0.0,35.743834,51.413290
75%,894.750000,2.000000,4.000000,4.000000,4.000000,4.000000,0.0,35.782519,51.447233
max,1193.000000,4.000000,5.000000,5.000000,5.000000,5.000000,0.0,38.097036,59.572999


In [65]:
df_cafe_1.to_csv('cafe_11.csv', index=False)

In [76]:
#fix data where is Nan
df_cafe_1['phone_number'] = df_cafe_1['phone_number'].fillna('0')
df_cafe_1.head()

,index,cafe_name,city,province,phone_number,cost,work_start,work_end,cafe_address,hookah,...,live_music,parking,pos,food_quality,service,cost_value,environment,follower,lat,lon
0,0,رولی (تبریز),tabriz,اول عباسی,04135243397,2,11:00:00,23:00:00,اول عباسی، به طرف منصور,False,...,False,False,True,0,0,0,0,0,38.080036,46.308585
1,1,لاله زار (تبریز),tabriz,چایکنار,04132820831,2,11:00:00,23:00:00,چایکنار، بعد از کارواش شهرداری، نرسیده به بلوا...,False,...,False,False,True,0,0,0,0,0,38.097036,46.271099
2,2,سانیز,tabriz,پل سنگی,04133346101,2,11:00:00,23:00:00,پل سنگی، چایکنار,False,...,False,False,True,0,0,0,0,0,38.077004,46.316582
3,3,آسا,tabriz,ولیعصر,04133314649,2,08:30:00,23:30:00,ولیعصر، خیابان شریعتی، نرسیده به میدان بهارستا...,False,...,False,False,True,0,0,0,0,0,38.064740,46.359343
4,4,کلبه (تبریز),tabriz,ولیعصر,04133249214,2,10:00:00,23:30:00,ولیعصر، نگین پارک,False,...,False,False,True,0,0,0,0,0,38.066835,46.351025


In [69]:

df_cafe_1.isna().sum()

index           0
cafe_name       0
city            0
province        0
phone_number    0
cost            0
work_start      0
work_end        0
cafe_address    0
hookah          0
internet        0
delivery        0
smoking         0
open_space      0
live_music      0
parking         0
pos             0
food_quality    0
service         0
cost_value      0
environment     0
follower        0
lat             0
lon             0
dtype: int64

In [43]:
#save data to Mysql
'''
from sqlalchemy import create_engine, Table, insert
import mysql.connector

user = 'user_group3'
password = 'hrsAWxaEJkIX_group3'
host = '45.139.10.138:80'
db = 'group3'

for index, row in df_cafe_1.iterrows():
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")
    print(engine.table_names())
    query_1="INSERT INTO  group3.cafe (cafe_name ,city ,province ,phone_number, cost, work_start, work_end) VALUES(%s,%s,%s,%s,%s,%s,%s)"
    my_data_1=(row['cafe_name'],  row['city'],  row['province'],  row['phone_number'],row['cost'], row['work_start'], row['work_end'])
    id=engine.execute(query_1,my_data_1)

    cafe_id = id.lastrowid
    print("ID of Row Added  = ",id.lastrowid)
    
    query_2="INSERT INTO  group3.cafe_address (cafe_id, cafe_address) VALUES(%s,%s)"
    my_data_2=(cafe_id, row['cafe_address'])
    id_2 = engine.execute(query_2,my_data_2)


    query_3="INSERT INTO  group3.cafe_rating (cafe_id, food_quality, service_quality, cost, cost_value, environment) VALUES(%s,%s,%s,%s,%s,%s)"
    my_data_3=(cafe_id, row['food_quality'], row['service'], row['cost'], row['cost_value'], row['environment'])
    id_3 = engine.execute(query_3,my_data_3)


    query_4="INSERT INTO  group3.cafe_features (cafe_id, hookah, internet, delivery, smoking, open_space, live_music, parking, pos) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    my_data_4=(cafe_id, row['hookah'], row['internet'], row['delivery'], row['smoking'], row['open_space'], row['live_music'], row['parking'], row['pos'])
    id_4 = engine.execute(query_4,my_data_4)
    '''

C:\Users\morte\AppData\Local\Temp\ipykernel_15140\3592569035.py:11: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  1
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  2
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  3
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  4
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  5
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  6
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  7
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  8
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  9
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  10
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  11
['cafe', 'cafe_address', 'cafe_features', 'cafe_rating']
ID of Row Added  =  12
['cafe', 'cafe_address', 'cafe_features', 'cafe_r